In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import time
import os
from pathlib import Path
import random
import socks
import socket

socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

def load_page(url):
    try:
        ua = UserAgent()
        headers = {'User-Agent': ua.random}
        page = requests.get(url, headers=headers)
        time.sleep(random.uniform(0.5,1.5))  
        soup = BeautifulSoup(page.content, 'html.parser')
        if soup is not None:
            return soup
        else:
            time.sleep(60)
    except requests.ConnectionError:
        print("Ошибка: Нет интернет-соединения.")
        time.sleep(600)
        return None
    except requests.RequestException as e:
        print(f"Ошибка при загрузке страницы {url}: {e}")
        return None

def get_match_info(match_url):
    soup = load_page(match_url)
    if soup is not None:
        if soup.find('div', class_='header-content-secondary'):
            datetime = soup.find('div', class_='header-content-secondary').find('time').get('datetime')
        else:
            time.sleep(10)
            return None
        try:
            team_1 = soup.find('div', class_='team-results').find('section', class_='radiant').find('span', class_='team-text team-text-full').text
        except:
            team_1 = 'Radiant'
        try:
            team_2 = soup.find('div', class_='team-results').find('section', class_='dire').find('span', class_='team-text team-text-full').text
        except:
            team_2 = 'Dire'
        try:
            try:
                result = soup.find('div', class_='match-result team radiant').find('span', class_="team-text team-text-full").text.strip()
            except:
                result = soup.find('div', class_='match-result team dire').find('span', class_="team-text team-text-full").text.strip()
        except:
            try:
                result = soup.find('div', class_='match-result team radiant').text.strip().replace(' Victory', '')
            except:
                result = soup.find('div', class_='match-result team dire').text.strip().replace(' Victory', '')
        heroes = []
        hero_elements = soup.find_all('div', class_='image-container image-container-hero image-container-icon')
        for hero_element in hero_elements:
            hero_name = hero_element.find('a').find('img').get('title').strip()
            heroes.append(hero_name)
        try:
            match_info = {
            'DateTime': datetime,
            'Team_Radiant': team_1,
            'Team_Dire': team_2,
            'Result': result,
            'DateTime': datetime,
            'Hero_Rad_1': heroes[0],
            'Hero_Rad_2': heroes[1],
            'Hero_Rad_3': heroes[2],
            'Hero_Rad_4': heroes[3],
            'Hero_Rad_5': heroes[4],
            'Hero_Dire_1': heroes[5],
            'Hero_Dire_2': heroes[6],
            'Hero_Dire_3': heroes[7],
            'Hero_Dire_4': heroes[8],
            'Hero_Dire_5': heroes[9]
        }
            return match_info
        except:
            return None
    else:
        return None


start_urls = [
             ]
page_urls = start_urls
for start_url in start_urls:
    soup = load_page(start_url)
    if soup is not None:
        page = soup.find('span', class_='next')
        try:
            if 'https://www.dotabuff.com' + page.find('a')['href'] in page_urls:
                continue
            else:
                page_urls.append('https://www.dotabuff.com' + page.find('a')['href'])
        except:
            continue
        
print(len(page_urls))

match_urls = []
for page in page_urls:
    soup = load_page(page)
    matchestd = soup.find_all('td', class_='cell-large')
    for matches in matchestd:
        matchesa = matches.find_all('a')
        for match in matchesa:
            if 'https://www.dotabuff.com' + match['href'] in match_urls:
                continue
            else:
                match_urls.append('https://www.dotabuff.com' + match['href'])
                
print(len(match_urls))

data = []
for match_url in match_urls:
    match_info = get_match_info(match_url)
    if match_info != None:
        data.append(match_info)
        if len(data) % 100 == 0:
            print(len(data))

df = pd.DataFrame(data, columns=['Team_Radiant', 'Team_Dire', 'Hero_Rad_1', 'Hero_Rad_2', 'Hero_Rad_3', 'Hero_Rad_4', 'Hero_Rad_5', \
                                 'Hero_Dire_1', 'Hero_Dire_2', 'Hero_Dire_3', 'Hero_Dire_4', 'Hero_Dire_5', 'DateTime', 'Result'])

desktop_path = str(Path.home() / "Desktop")

df.to_csv(os.path.join(desktop_path, "dota2dataset.csv"), index=False)


print(df.head(), df.shape)